In [1]:
import os
import pickle
import collections
import datetime
import copy
import numpy as np
import scipy as sp
import pandas as pd
import happybase
import tensorflow as tf
import warnings
import csv
import re
import json
import sklearn.model_selection
import sklearn.preprocessing

In [2]:
data_dir = r"/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data"

In [3]:
file_list = []
for root,_,path_list in os.walk(data_dir):
    for path in path_list:
        file_list.append(os.path.join(root,path))
print(file_list)

['/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000000_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000002_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000001_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000003_0']


In [4]:
%%time
#读取用户数据
all_file_data = []
for file_name in file_list:
    all_file_data.append(pd.read_csv(os.path.join(data_dir,file_name),sep='\001',header=None,error_bad_lines=False,na_values=["-1",-1]))
df = pd.concat(all_file_data,axis=0,ignore_index=True)
del all_file_data
column_list = ['用户id','用户姓名','用户性别','用户出生时间','所在公司','所处职位','行业','微信网名','微信性别','常驻省','常驻市','是否付费用户','付费等级','关注的作者','作者类型','关注的马甲','关注的公司','文章id','是否精选文章','浏览文章时间','浏览时长','文章频道','文章类型','文章标签','文章图片数量','文章字数','新定位省','新定位市','评论内容','打赏金额','设备型号','使用网络']
df.columns = column_list

b'Skipping line 461777: expected 32 fields, saw 33\n'
<string>:2: DtypeWarning: Columns (1,3,4,5,6,7,15,16,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
<string>:2: DtypeWarning: Columns (1,2,3,4,5,6,7,15,16,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 54.9 s, sys: 3.33 s, total: 58.3 s
Wall time: 58.3 s


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15703265 entries, 0 to 15703264
Data columns (total 32 columns):
 #   Column  Dtype  
---  ------  -----  
 0   用户id    object 
 1   用户姓名    object 
 2   用户性别    object 
 3   用户出生时间  object 
 4   所在公司    object 
 5   所处职位    object 
 6   行业      object 
 7   微信网名    object 
 8   微信性别    float64
 9   常驻省     float64
 10  常驻市     float64
 11  是否付费用户  float64
 12  付费等级    float64
 13  关注的作者   float64
 14  作者类型    float64
 15  关注的马甲   object 
 16  关注的公司   object 
 17  文章id    object 
 18  是否精选文章  object 
 19  浏览文章时间  object 
 20  浏览时长    float64
 21  文章频道    object 
 22  文章类型    object 
 23  文章标签    object 
 24  文章图片数量  object 
 25  文章字数    object 
 26  新定位省    object 
 27  新定位市    object 
 28  评论内容    object 
 29  打赏金额    object 
 30  设备型号    object 
 31  使用网络    object 
dtypes: float64(8), object(24)
memory usage: 3.7+ GB


In [6]:
df.isna().sum()

用户id             0
用户姓名      15281129
用户性别      12543401
用户出生时间    12543401
所在公司      15493250
所处职位      15164135
行业        15539019
微信网名      15616490
微信性别      12543403
常驻省       15703265
常驻市       15703265
是否付费用户    15358817
付费等级      15358817
关注的作者     14981704
作者类型      14981704
关注的马甲     15020605
关注的公司     15483640
文章id       3492415
是否精选文章     4212930
浏览文章时间     3159947
浏览时长      12198198
文章频道       6485058
文章类型      10853406
文章标签      13521462
文章图片数量     6214475
文章字数       6214475
新定位省       3159946
新定位市       3159945
评论内容      15683678
打赏金额      15703250
设备型号      15064728
使用网络       3159945
dtype: int64

'359998,376736,375646,376887,374409,375982'

In [8]:
df["文章id"]

0                   359998,376736,375646,376887,374409,375982
1           343666,350648,349454,348559,349190,347319,3494...
2                                                      377404
3           373301,361213,361120,373288,361277,361175,3732...
4           364037,363419,363534,363337,364246,364164,3633...
                                  ...                        
15703260                                               265074
15703261                                               108013
15703262                                               363341
15703263                                               299101
15703264                                               217531
Name: 文章id, Length: 15703265, dtype: object

In [15]:
df["行业"]

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
           ... 
15703260    NaN
15703261    NaN
15703262    NaN
15703263    NaN
15703264    NaN
Name: 行业, Length: 15703265, dtype: object

In [9]:
age_df = pd.read_csv(r"/mnt1/zhaodachuan/data/predict_user_attribute/data/age.csv")

In [14]:
age_df["age"]

0           30~35
1           30~35
2           30~35
3           30~35
4           30~35
            ...  
15703260    30~35
15703261    30~35
15703262    30~35
15703263    30~35
15703264    30~35
Name: age, Length: 15703265, dtype: object

In [10]:
position_df = pd.read_csv(r"/mnt1/zhangwanli/data/position.csv")

In [16]:
position_df["position"]

0           事业单位员工
1           事业单位员工
2           事业单位员工
3           事业单位员工
4           事业单位员工
             ...  
15703260    事业单位员工
15703261    事业单位员工
15703262    事业单位员工
15703263    事业单位员工
15703264    事业单位员工
Name: position, Length: 15703265, dtype: object

In [20]:
df['用户id']

0                   00001D26FE991A4BC4A01325D5E222DE
1                   00006C3EF7F79B0F37EEFB8381505530
2                   0000D0E1960239E8362F6B4100D9790A
3           0000f55e60d5ba7b36004429b235c95096f7153d
4                   00014E9D5CF948B3FE6376564B23F64F
                              ...                   
15703260                        qlu2xsthqk7tp0mncpgr
15703261                        qlu30bysbp1mj150uc1m
15703262                         qlu30t7n6f2b1jgzexv
15703263                        qlu315b37v5rc273amb0
15703264                         qlu31nan912a53u0z62
Name: 用户id, Length: 15703265, dtype: object

In [ ]:
本科及以上
大专
高中
其他

In [17]:
df["行业"].unique()

array([nan, '内容/营销/传播', '教育/专业服务/培训', '创业者', '互联网', '电商/仓储物流', '投资人',
       'IT', 'TMT从业人员', '商业服务', '酒店/餐饮/旅游', '金融', '政府/非盈利机构',
       '企业服务/云计算/大数据', '房地产', '汽车/出行', '媒体', '加工制造', '其他', '制药医疗/生物/卫生保健',
       '文化/体育/娱乐业', '商业服务-O2O/服务/社区', '人工智能/前沿科技', '法律', '智能硬件/硬件制造', '0',
       '贸易/零售批发', '学生', '能源相关', 'undefined', '农业', '仓储物流', '零售/快消', 0.0,
       '营销'], dtype=object)

In [18]:
age_df["age"].unique()

array(['30~35', '26~29', '36~45', '~20', '21~25', '46~50', '50~'],
      dtype=object)

In [19]:
position_df["position"].unique()

array(['事业单位员工', '企业业务决策层', '企业普通员工', '企业创始人/CEO', '企业一般管理人员', '高校学生'],
      dtype=object)

In [41]:

college_set = set()
high_school_set = set()
junior_college_set = set()

for uuid,trade,age,position,aid_list in zip(df['用户id'],df["行业"],age_df["age"],position_df["position"],df["文章id"]):
    education = "其他"
    if trade == trade and (not (trade in set(['undefined',"农业","加工制造","酒店/餐饮/旅游","商业服务","商业服务-O2O/服务/社区","文化/体育/娱乐业","其他","仓储物流","零售/快消"]))):
        education = "本科"
        if aid_list == aid_list:
            college_set.update(str(aid_list).split(","))
    elif trade == trade and trade in set(["农业","加工制造","酒店/餐饮/旅游","商业服务-O2O/服务/社区","商业服务","仓储物流","零售/快消"]):
        education = "高中"
        if aid_list == aid_list:
            high_school_set.update(str(aid_list).split(","))
    elif trade == trade and trade in set(["文化/体育/娱乐业","其他"]):
        education = "大专"
        if aid_list == aid_list:
            junior_college_set.update(str(aid_list).split(","))
    else:
        if position in set(['企业业务决策层', '企业普通员工', '企业创始人/CEO', '企业一般管理人员', '高校学生']):
            if aid_list == aid_list:
                college_set.update(str(aid_list).split(","))
            

In [42]:
res = {"uuid_list":[],"education_list":[]}

In [43]:
for uuid,trade,age,position,aid_list in zip(df['用户id'],df["行业"],age_df["age"],position_df["position"],df["文章id"]):
    education = "其他"
    if trade == trade and (not (trade in set(['undefined',"农业","加工制造","酒店/餐饮/旅游","商业服务","商业服务-O2O/服务/社区","文化/体育/娱乐业","其他","仓储物流","零售/快消"]))):
        education = "本科"
    elif trade == trade and trade in set(["农业","加工制造","酒店/餐饮/旅游","商业服务-O2O/服务/社区","商业服务","仓储物流","零售/快消"]):
        education = "高中"
    elif trade == trade and trade in set(["文化/体育/娱乐业","其他"]):
        education = "大专"
    else:
        if aid_list == aid_list:
            aid_set = set(aid_list.split(","))
            if aid_set.issubset(college_set):
                education = "本科"
            elif aid_set.issubset(high_school_set):
                education = "高中"
            elif aid_set.issubset(junior_college_set):
                education = "大专"
        if position in set(['企业业务决策层', '企业普通员工', '企业创始人/CEO', '企业一般管理人员', '高校学生']):
            education = "本科"
    res["uuid_list"].append(uuid)
    res["education_list"].append(education)

In [44]:
collections.Counter(res["education_list"])

Counter({'其他': 15508728, '本科': 181857, '高中': 7253, '大专': 5427})

In [31]:
college_set

set()

In [24]:
collections.Counter(res["education_list"])

Counter({'其他': 15539020, '本科': 151565, '高中': 7253, '大专': 5427})

In [46]:
res["udid_list"] = res["uuid_list"]

In [49]:
pool = happybase.ConnectionPool(size=100, host="47.110.125.69", port=9090, timeout=None,
                                protocol='binary'
                                )
with pool.connection() as conn:
    table = conn.table("feature_center:user_profile")
    with table.batch(batch_size=400) as batch:
        for i,(udid,education) in enumerate(zip(res["udid_list"],res["education_list"])):
            batch.put(udid.encode(), {"attribute:predict_education".encode(): education.encode()})
            #print(udid.encode(), {"attribute:predict_education".encode(): education.encode()})
            #break
            if i % 100000 == 0:
                print(i)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000